<a href="https://colab.research.google.com/github/AbdulxoliqMirzayev/Anomaly-Detection-and-Energy-Optimization-System/blob/main/Copy_of_anomaly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
import gradio as gr
import smtplib
from email.mime.text import MIMEText
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
from google.colab import files


In [3]:

# Loyiha maqsadi
def project_description():
    return """
    Bu loyiha ishlab chiqarish jarayonida uskunalarning ishlashini kuzatish va energiya sarfini optimallashtirish uchun sun'iy intellektga asoslangan tizimdir.
    Ushbu tizim ishlab chiqarishdagi anomaliyalarni aniqlash, texnik xizmat ko'rsatishni tavsiya qilish orqali uskunalar samaradorligini oshirishga yordam beradi.
    Loyiha energiya sarfini kamaytirish, nosozliklarni erta aniqlash va umumiy ishlab chiqarish samaradorligini oshirishga qaratilgan. Tizim anomaliyalarni aniqlash, energiya sarfini tahlil qilish, va uskunalar orasidagi o'xshashliklarni baholash imkoniyatlarini taqdim etadi.
    """


In [4]:
# fayil yuklash funksiyasi
uploaded = files.upload()
file_path = list(uploaded.keys())[0]

def load_dataset(file_path):
    try:
        data = pd.read_csv(file_path)
        return data
    except FileNotFoundError:
        return None

data = load_dataset(file_path)
if data is None:
    print(f"Fayl topilmadi: {file_path}. Iltimos, fayl manzilini tekshiring.")
    exit()


Saving predictive_maintenance (1).csv to predictive_maintenance (1).csv


In [5]:
# Energiyani hisoblash ***

data['Energy Consumption [kWh]'] = (
    data['Rotational speed [rpm]'] * data['Torque [Nm]'] * data['Process temperature [K]']
) / 10000

# Uskunaga ID qo'shish ****
data.reset_index(inplace=True)
data.rename(columns={'index': 'Equipment ID'}, inplace=True)

# Anomaliyalarni aniqlash uchun kerakli ustunlarni tanlaymiz ****
features = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']
X = data[features]


In [6]:
# IsolationForest modelini yaratish va orgatish
isolation_forest = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
isolation_forest.fit(X)

data['Anomaly'] = isolation_forest.predict(X)

X_train, X_test, y_train, y_test = train_test_split(X, data['Anomaly'], test_size=0.2, random_state=42)

# AUC orqali modelni baholash
y_pred_test_score = isolation_forest.decision_function(X_test)
roc_auc = roc_auc_score(y_test == 1, y_pred_test_score)


In [7]:
# Foydalanuvchidan texnik yordamchi email manzilini kiritishini so'rash funksiyasi
def get_technician_email():
    return "Texnik yordamchi email manzilini kiriting:"


In [8]:
# Anomaliyalar tahlili uchun funksiya
def analyze_anomalies(data):
    anom_count = data['Anomaly'].value_counts()
    danomalies = data[data['Anomaly'] == -1]
    anomaly_details = danomalies[['Equipment ID', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Energy Consumption [kWh]']]
    return anom_count, anomaly_details


In [9]:
# Anomaliyaga uchragan va uchramagan uskunani taqqoslash

def compare_anomalous_vs_normal(data):
    danomalies = data[data['Anomaly'] == -1]
    normal = data[data['Anomaly'] == 1]
    anomalous_sample = danomalies.iloc[0]
    normal_sample = normal.iloc[0]
    comparison = pd.DataFrame({
        'Feature': ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Energy Consumption [kWh]'],
        'Anomalous Equipment': [
            anomalous_sample['Air temperature [K]'],
            anomalous_sample['Process temperature [K]'],
            anomalous_sample['Rotational speed [rpm]'],
            anomalous_sample['Torque [Nm]'],
            anomalous_sample['Tool wear [min]'],
            anomalous_sample['Energy Consumption [kWh]']
        ],
        'Normal Equipment': [
            normal_sample['Air temperature [K]'],
            normal_sample['Process temperature [K]'],
            normal_sample['Rotational speed [rpm]'],
            normal_sample['Torque [Nm]'],
            normal_sample['Tool wear [min]'],
            normal_sample['Energy Consumption [kWh]']
        ]
    })
    return comparison

# Texnik yordamchiga xabar berish
anomaly_count, anomaly_details = analyze_anomalies(data)


In [10]:
# Gradio interfeysini yaratamiz *** --- ***

def main_interface():
    description = project_description()
    email_prompt = get_technician_email()
    comparison_df = compare_anomalous_vs_normal(data)

    return description, roc_auc, email_prompt, comparison_df

description, roc_auc, email_prompt, comparison_df = main_interface()

def gradio_app():
    with gr.Blocks() as demo:
        gr.Markdown("## Anomaly Detection and Maintenance Recommendation System")
        gr.Markdown(description)

        roc_auc_text = f"AUC score: {roc_auc}"
        gr.Textbox(value=roc_auc_text, label="Model Evaluation", interactive=False)

        gr.Markdown("### Anomalous vs Normal Equipment Comparison")
        gr.Dataframe(value=comparison_df)

        gr.Markdown("### Anomalies vs Normal Equipment (Scatter Plot)")
        scatter_plot_btn = gr.Button("Grafikni ko'rish")
        def plot_scatter():
            sample_size = 300
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.scatter(data[data['Anomaly'] == -1]['Rotational speed [rpm]'].sample(n=min(sample_size, len(data[data['Anomaly'] == -1]))),
                       data[data['Anomaly'] == -1]['Torque [Nm]'].sample(n=min(sample_size, len(data[data['Anomaly'] == -1]))),
                       color='red', label='Anomalies')
            ax.scatter(data[data['Anomaly'] == 1]['Rotational speed [rpm]'].sample(n=min(sample_size, len(data[data['Anomaly'] == 1]))),
                       data[data['Anomaly'] == 1]['Torque [Nm]'].sample(n=min(sample_size, len(data[data['Anomaly'] == 1]))),
                       color='green', label='Normal')
            ax.set_xlabel('Rotational speed [rpm]')
            ax.set_ylabel('Torque [Nm]')
            ax.set_title('Anomalies vs Normal Equipment (Scatter Plot)')
            ax.legend()
            plt.savefig("scatter_plot.png")
            return "scatter_plot.png"
        scatter_plot_img = gr.Image(type="filepath")
        scatter_plot_btn.click(plot_scatter, outputs=scatter_plot_img)
        gr.Markdown("Bu grafik anomaliyalar va normal uskunalarning moment kuchi va aylanish tezligini taqqoslaydi. Qizil nuqtalar anomaliyalarni, yashil nuqtalar esa normal holatdagi uskunalarni ifodalaydi. Bu grafikdan foydalanuvchi anomaliyalar va normal holatlarning qanday farq qilishini ko'rishi mumkin.")

        gr.Markdown("### Anomalies vs Normal Equipment (Pie Chart)")
        pie_chart_btn = gr.Button("Grafikni ko'rish")
        def plot_pie():
            anomaly_counts = [len(data[data['Anomaly'] == -1]), len(data[data['Anomaly'] == 1])]
            labels = ['Anomalies', 'Normal']
            colors = ['#FF6347', '#4682B4']
            fig, ax = plt.subplots(figsize=(8, 8))
            ax.pie(anomaly_counts, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90, wedgeprops={'edgecolor': 'black'})
            ax.set_title('Distribution of Anomalies vs Normal Equipment (Pie)', fontsize=16)
            plt.savefig("pie_chart.png")
            return "pie_chart.png"
        pie_chart_img = gr.Image(type="filepath")
        pie_chart_btn.click(plot_pie, outputs=pie_chart_img)
        gr.Markdown("Bu diagramma uskunalar orasida qancha qismi anomaliya va qancha qismi normal ekanligini ko'rsatadi. Diagrammadagi foizlar orqali anomaliyalar va normal uskunalar nisbati haqida umumiy tasavvur hosil qilish mumkin.")

        # Anomaliyaga uchragan uskunaning energiya sarfini grafikda chiqarish bolimi *******

        gr.Markdown("### Anomalous Equipment Energy Consumption")
        anomaly_energy_btn = gr.Button("Anomalous Equipment Energy Consumption Grafikni ko'rish")
        def plot_anomaly_energy():
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.plot(data[data['Anomaly'] == -1].index, data[data['Anomaly'] == -1]['Energy Consumption [kWh]'], label='Anomalies - Energy Consumption [kWh]', color='red')
            ax.set_xlabel('Index')
            ax.set_ylabel('Energy Consumption [kWh]')
            ax.set_title('Anomalous Equipment Energy Consumption')
            ax.legend()
            plt.savefig("anomaly_energy.png")
            return "anomaly_energy.png"
        anomaly_energy_img = gr.Image(type="filepath")
        anomaly_energy_btn.click(plot_anomaly_energy, outputs=anomaly_energy_img)
        gr.Markdown("Bu grafik anomaliyaga uchragan uskunalarning energiya sarfini ko'rsatadi. Qizil chiziq orqali anomaliyalar energiya sarfi vaqt bo'yicha qanday o'zgarayotganini kuzatishingiz mumkin.")

        # Anomaliyaga uchramagan uskunalarning energiya sarfini grafikda chiqarish *****

        gr.Markdown("### Normal Equipment Energy Consumption (Smoothed, Sample)")
        normal_energy_btn = gr.Button("Normal Equipment Energy Consumption Grafikni ko'rish")
        def plot_normal_energy():
            normal_rolling = data[data['Anomaly'] == 1]['Energy Consumption [kWh]'].rolling(window=150).mean()
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.plot(data[data['Anomaly'] == 1].index[:300], normal_rolling[:300], label='Normal Equipment - Energy Consumption (Rolling Average)', color='green')
            ax.set_xlabel('Index')
            ax.set_ylabel('Energy Consumption [kWh]')
            ax.set_title('Normal Equipment Energy Consumption (Smoothed, Sample)')
            ax.legend()
            plt.savefig("normal_energy.png")
            return "normal_energy.png"
        normal_energy_img = gr.Image(type="filepath")
        normal_energy_btn.click(plot_normal_energy, outputs=normal_energy_img)
        gr.Markdown("Bu grafik normal holatdagi uskunalarning energiya sarfini ko'rsatadi. Yashil chiziq energiya sarfining silliqlangan qiymatlarini ifodalaydi, bu esa trendlarni yaxshiroq tushunishga yordam beradi.")

        # Anomaliyalarni tekshirish va sababini chiqarish ****

        gr.Markdown("### Anomaliyalarni Tekshirish")
        check_anomalies_btn = gr.Button("Anomaliyalarni Tekshirish")
        anomaly_output_box = gr.Dataframe(label="Anomaliya Tafsilotlari")
        anomaly_reason_box = gr.Markdown()

        def check_anomalies():
            if not anomaly_details.empty:
                selected_anomaly = anomaly_details.sample(1)
                reasons = []
                if selected_anomaly['Air temperature [K]'].values[0] > data['Air temperature [K]'].mean():
                    reasons.append("Havo harorati yuqori bo'lishi anomaliyaga sabab bo'lishi mumkin.")
                if selected_anomaly['Process temperature [K]'].values[0] > data['Process temperature [K]'].mean():
                    reasons.append("Jarayon harorati yuqori bo'lishi anomaliyaga sabab bo'lishi mumkin.")
                if selected_anomaly['Energy Consumption [kWh]'].values[0] > data['Energy Consumption [kWh]'].mean():
                    reasons.append("Energiya sarfining oshishi anomaliyaga olib kelishi mumkin.")
                if selected_anomaly['Torque [Nm]'].values[0] > data['Torque [Nm]'].mean():
                    reasons.append("Moment kuchining oshishi nostandart ishlashga sabab bo'lishi mumkin.")
                reason_text = "<br>".join(reasons) if reasons else "Anomaliyaga olib keladigan hech qanday maxsus sabab topilmadi."
                return selected_anomaly, reason_text
            else:
                return pd.DataFrame(["Anomaliyalar topilmadi."], columns=["Xabar"]), ""

        check_anomalies_btn.click(fn=check_anomalies, outputs=[anomaly_output_box, anomaly_reason_box])

        # Anomaliyaga yaqin uskunalarni tekshirish
        gr.Markdown("### Anomaliyaga Yaqin Uskunalarni Tekshirish")
        check_near_anomalies_btn = gr.Button("Anomaliyaga Yaqin Uskunalarni Tekshirish")
        near_anomaly_output_box = gr.Dataframe(label="Anomaliyaga Yaqin Uskunalar Tafsilotlari")
        near_anomaly_reason_box = gr.Markdown()

        def check_near_anomalies():
            if not anomaly_details.empty:
                selected_anomaly = anomaly_details.sample(1)
                distances = euclidean_distances(X, selected_anomaly[features])
                data['Distance'] = distances
                sorted_data = data.sort_values(by='Distance')
                near_anomalies = sorted_data[(sorted_data['Anomaly'] == 1) & (sorted_data['Distance'] < sorted_data['Distance'].quantile(0.05))]
                if len(near_anomalies) > 3:
                    near_anomalies = near_anomalies.head(3)
                if not near_anomalies.empty:
                    near_anomalies_info = near_anomalies[['Equipment ID', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Energy Consumption [kWh]', 'Distance']]
                    reasons = []
                    if (near_anomalies_info['Air temperature [K]'] > selected_anomaly['Air temperature [K]'].values[0]).any():
                        reasons.append("Havo harorati yuqori bo'lishi anomaliyaga sabab bo'lishi mumkin.")
                    if (near_anomalies_info['Energy Consumption [kWh]'] > selected_anomaly['Energy Consumption [kWh]'].values[0]).any():
                        reasons.append("Energiya sarfining oshishi anomaliyaga olib kelishi mumkin.")
                    if (near_anomalies_info['Process temperature [K]'] > selected_anomaly['Process temperature [K]'].values[0]).any():
                        reasons.append("Jarayon haroratining oshishi anomaliyaga olib kelishi mumkin.")
                    reason_text = "<br>".join(reasons) if reasons else "Anomaliyaga olib keladigan hech qanday maxsus sabab topilmadi."
                    return near_anomalies_info, reason_text
                else:
                    return pd.DataFrame(["Anomaliyaga yaqin uskunalar topilmadi."]), ""
            else:
                return pd.DataFrame(["Anomaliyalar topilmadi."]), ""

        check_near_anomalies_btn.click(fn=check_near_anomalies, outputs=[near_anomaly_output_box, near_anomaly_reason_box])

    demo.launch()

gradio_app()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b69137b760903ff205.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
